In [2]:
import onshape.brepio as brepio
import meshplot as mp
import numpy as np
from visualize import inspect
import os
from utils import adjacency_matrix, connected_components
import pandas
from sklearn.neighbors import NearestNeighbors
import pandas as pd
from pspart import Part
import pspart
import numpy.linalg as LA
from IPython.display import clear_output
import pandas as ps
from intervaltree import IntervalTree, Interval

## Initialization

In [3]:
datadir = '/projects/grail/benjones/cadlab'

In [4]:
loader = brepio.Loader(datadir)

In [5]:
with open('filter_list.txt') as f:
    filter_list = f.readlines()
print('making set')
filter_set = set()
for l in filter_list:
    filter_set.add(l.strip())
del filter_list

making set


In [6]:
datalist = [entry.name for entry in os.scandir('/projects/grail/benjones/cadlab/data/flattened_assemblies') if entry.name not in filter_set]

In [7]:
index = [f for f in enumerate(datalist) if f[1].startswith('7e9d103329425a0159111b74_0adf8a89c08f3d158c92fe1d_000e8d5b205966be9b869cc7')][0][0]

In [8]:
name = datalist[index]
print(name)
geo, mates = loader.load_flattened(name)

7e9d103329425a0159111b74_0adf8a89c08f3d158c92fe1d_000e8d5b205966be9b869cc7.json


In [8]:
mateIndex = 0
currMate = mates[mateIndex]
occs = [me[0] for me in currMate.matedEntities]

In [9]:
parts = [geo[occ] for occ in occs]

## Testing stuff

In [249]:
parts[0][1].get_example_mc(parts[0][1].onshape_ids[0])

IndexError: list index out of range

In [254]:
parts[0][1]

<Part with 0 nodes and 0 links>

In [ ]:
parts[0][1].get_onshape_def_from_mc(parts[0][1].all_mate_connectors[0])

In [ ]:
parts[0][1].get_example_mc(parts[0][1].onshape_ids[0])

In [ ]:
dir(parts[0][1].all_mate_connectors[0])

In [ ]:
dir(parts[0][1].all_mate_connectors[0].location_inference.inference_type)

## Construct example for network

In [9]:
maxdim = max([(part[1].bounding_box()[1]-part[1].bounding_box()[0]).max() for part in parts])

NameError: name 'parts' is not defined

In [260]:
locations = [np.array([part[0][:3,:3] @ mc.get_coordinate_system()[:3,3] + part[0][:3,3] for mc in part[1].all_mate_connectors]) for part in parts]

In [261]:
import meshplot as mp

In [262]:
p = mp.plot(locations[0])
p.add_points(locations[1], shading={'point_color':'blue'})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.081188…

1

In [14]:
[loc.shape[0] for loc in locations]

[150, 433]

In [15]:
startIndex = np.argmax([loc.shape[0] for loc in locations])

In [16]:
nn = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(locations[startIndex])

In [17]:
distances, indices = nn.kneighbors(locations[1-startIndex])

In [18]:
threshold = maxdim/10000
matches = [(i,pair[1][0],pair[0][0]) for i,pair in enumerate(zip(distances, indices)) if pair[0] < threshold]

In [19]:
uniqueIndices = set()
for match in matches:
    uniqueIndices.add(match[1])

bestmatches = [min([match for match in matches if match[1] == index], key=lambda a: a[2]) for index in uniqueIndices]

In [20]:
matches

[(1, 354, 1.734723475976807e-17),
 (3, 354, 2.5018537765542006e-17),
 (6, 354, 2.6020852139652106e-17),
 (28, 354, 2.5018537765542006e-17),
 (30, 354, 2.5018537765542006e-17),
 (81, 354, 2.5018537765542006e-17),
 (82, 354, 2.5018537765542006e-17)]

In [21]:
bestmatches

[(1, 354, 1.734723475976807e-17)]

In [9]:
from automate.lightning_models.simplified import SimplifiedJointModel
from extension.ml import Predictor

In [10]:
from automate.data.data import UniformMateData
from torch_geometric.data import Batch

In [11]:
import torch

In [12]:
predictor_types = Predictor('/projects/grail/benjones/cadlab/dalton_lightning_logs/real_all_fn_args_amounts_sum_directedhybridgcn12_type/version_0/checkpoints/epoch=5-val_auc=0.948979.ckpt')
predictor_location = Predictor('/projects/grail/benjones/cadlab/dalton_lightning_logs/real_all_fn_args_amounts_sum_directedhybridgcn12/version_0/checkpoints/epoch=46-val_auc=0.666113.ckpt')

/fast/jamesn8/anaconda3/envs/torch3/lib/python3.9/site-packages/deprecate/deprecation.py:115: LightningDeprecationWarning: The `Metric` was deprecated since v1.3.0 in favor of `torchmetrics.metric.Metric`. It will be removed in v1.5.0.
  stream(template_mgs % msg_args)


### Find likely mates

In [13]:
#parts = list(reversed([geo[occ] for occ in geo]))
parts = [geo[occ] for occ in geo]

In [14]:
epsilon_fac = 1000
maxdim = max([(part.bounding_box()[1]-part.bounding_box()[0]).max() for _, part in parts])
epsilon = maxdim/epsilon_fac
print(f'maxdim:{maxdim}, epsilon:{epsilon}')
mc_locations = []
interval2part = []
part2offset = dict()
total_mcs = 0
for i,tf_part in enumerate(parts):
    tf, part = tf_part
    for mc in part.all_mate_connectors:
        cs = mc.get_coordinate_system()
        origin = tf[:3, :3] @ cs[:3,3] + tf[:3,3]
        mc_locations.append(origin)
    new_total = total_mcs + len(part.all_mate_connectors)
    interval2part.append((total_mcs, new_total, i))
    part2offset[i] = total_mcs
    total_mcs = new_total
nnhash = pspart.NNHash(mc_locations, 3, epsilon)
tree = IntervalTree([Interval(l, u, d) for l, u, d in interval2part])

maxdim:0.10435554761121174, epsilon:0.00010435554761121175


In [15]:
proposals = set()
num_matches = 0
for i,loc in enumerate(mc_locations):
    nearest = list(nnhash.get_nearest_points(loc))
    num_matches += len(nearest)
    part_index = next(iter(tree[i])).data
    for j in nearest:
        other_part_index = next(iter(tree[j])).data
        if other_part_index != part_index:
            pi1, pi2 = part_index, other_part_index
            mci1, mci2 = i - part2offset[part_index], j - part2offset[other_part_index]
            if pi1 > pi2:
                pi1, pi2 = pi2, pi1
                mci1, mci2 = mci2, mci1
            proposals.add((pi1, pi2, mci1, mci2))

In [16]:
len(proposals)

28

In [17]:
results = predictor_location.predict_assembly([part[1] for part in parts], proposals)

In [19]:
results[0][0].item()

0.9689185619354248

### Create batch

In [18]:
parts = [geo[occ][1] for occ in geo]
part_topology_offsets = dict()
topo_offset = 0
for i,part in enumerate(parts):
    part_topology_offsets[i] = topo_offset
    topo_offset += part.num_topologies
transforms = predictor_location.transform
follow_batch = [
            'node_types_g1', 'node_types_g2',
            'mc_index_g1', 'mc_index_g2',
            'mc_pair_labels',
            'left_mc_individual_labels', 'right_mc_individual_labels'
        ]

max_size = 0
for part in parts:
    p_size = (part.V.max(axis=0) - part.V.min(axis=0)).max()
    max_size = max(max_size, p_size)

uniform_data = []
for part in parts:
    or1, loc1, inf1 = part.get_example_mc(part.onshape_ids[0])
    p_center = (part.V.max(axis=0) + part.V.min(axis=0))/2
    p_normalized = np.identity(4, dtype=float)
    p_normalized[:3,3] = -p_center
    p_normalized[3,3] = max_size
    data = UniformMateData(
        part,
        or1,
        loc1,
        inf1,
        p_normalized,
        part,
        or1,
        loc1,
        inf1,
        p_normalized,
        'FASTENED'
    )
    data = transforms(data)
    uniform_data.append(data)

batch = Batch.from_data_list(uniform_data, follow_batch=follow_batch)
mc_pairs = []
for p in proposals:
    mc1 = parts[p[0]].all_mate_connectors[p[2]]
    mc2 = parts[p[1]].all_mate_connectors[p[3]]
    offset1 = part_topology_offsets[p[0]]
    offset2 = part_topology_offsets[p[1]]
    mc = torch.tensor([mc1.orientation_inference.topology_ref + offset1, mc1.location_inference.topology_ref + offset1, mc1.location_inference.inference_type.value,
                       mc2.orientation_inference.topology_ref + offset2, mc2.location_inference.topology_ref + offset2, mc2.location_inference.inference_type.value], dtype=torch.int64)
    mc_pairs.append(mc.unsqueeze(-1))
mc_pairs = torch.hstack(mc_pairs)

In [19]:
batch.mc_pairs = mc_pairs

In [48]:
for k in follow_batch:
    if k not in data.__dict__:
        print(k)

mc_index_g1
mc_index_g2


In [ ]:
results = []
for match in bestmatches:
    parts_reordered = (parts[1-startIndex], parts[startIndex]) #query index, NN tree index
    
    selections = [{'part':part[1],'mc':part[1].all_mate_connectors[match[i]]} for i,part in enumerate(parts_reordered)]    
    result = predictor.predict_mc(selections)
    results.append(result)

In [35]:
results

[(tensor([[0.7990, 0.3978, 0.2965, 0.2671, 0.0985, 0.0746, 0.0728, 0.0220]],
         grad_fn=<SigmoidBackward>),
  tensor([[[  0,   0,   0,   0,   0,   0,   0,   0]],
  
          [[  0,   0,   0,   0,   0,   0,   0,   0]],
  
          [[  4,   4,   4,   4,   4,   4,   4,   4]],
  
          [[ 18,  18,  18,  18,  18,  18,  18,  18]],
  
          [[110, 159, 160, 109, 107, 108, 161,  18]],
  
          [[  2,   3,   3,   2,   0,   2,   3,   1]]]))]

In [239]:
def predict_mc(predictor, selections):
        assert(len(selections) == 2)
        for sel in selections:
            assert('part' in sel)
            assert('mc' in sel)
            #assert('occurrence' in sel)
        
        part1 = selections[0]['part']
        or1, loc1, inf1 = part1.get_onshape_def_from_mc(selections[0]['mc'])
        part2 = selections[1]['part']
        or2, loc2, inf2 = part2.get_onshape_def_from_mc(selections[1]['mc'])

        data = UniformMateData(
            selections[0]['part'],
            or1,
            loc1,
            inf1,
            None,
            selections[1]['part'],
            or2,
            loc2,
            inf2,
            None,
            'FASTENED'
        )
        data = predictor.transform(data)


        matepair = []
        for sel in selections:
            matepair.append(sel['mc'].orientation_inference.topology_ref)
            matepair.append(sel['mc'].location_inference.topology_ref)
            matepair.append(sel['mc'].location_inference.inference_type.value)

        found = False
        pairIndex = -1
        for i in range(data.mc_pairs.shape[1]):
            col = data.mc_pairs[:,i].numpy()
            if all(col == np.array(matepair)):
                found = True
                pairIndex = i
                break
        assert(found)
        data.mc_pair_labels[:] = 0
        data.mc_pair_labels[pairIndex] = 1

        batch = Batch.from_data_list([data], follow_batch=predictor.follow_batch)
        predictor.model.eval()
        preds = predictor.model(batch)
        #sorted_mates = batch.mc_pairs[:,indices]
        #sorted_left_cs = batch.mc_pairs_left_cs[indices,:]
        #sorted_right_cs = batch.mc_pairs_right_cs[indices,:]
        return torch.sigmoid(preds)#, torch.transpose(sorted_left_cs.reshape((-1,4,4)), 1, 2), torch.transpose(sorted_right_cs.reshape((-1,4,4)),1,2)

In [240]:
result = predict_mc(predictor, selections)

NameError: name 'predictor' is not defined